<h1>REMOVE PUTATIVE CONTAMINATION READS</h1>
<p>These steps are set to remove reads that are flagged as potential contaminators.
<ol>
<li>Blast contigs to nt Database.</li>
<li>Mapping sequencing reads to <i>de novo</i> assembly.</li>
<li>Parse blast results.</li>
<li>Identify contigs to be removed.</li>
<li>Identify reads to be removed.</li>
<li>Removed flagged reads.</li>
</ol>
The programs Blast<sup>1</sup> and BWA<sup>2</sup> are required to complete these steps. The results are located at:
<ul>
<li>Decontaminated seequencing reads can be found in <b>results/decontamination/&lt;ISOLATE_ID&gt;_1.dcn.fq</b> & <b>results/decontamination/&lt;ISOLATE_ID&gt;_2.dcn.fq</b> </li>
<li>Summary of reads removed can be found in <b>results/decontamination/ContaminationRemovalSummary.tsv</b></li>
<li>Decontaminated assemblies can be found in <b>results/spades/&lt;ISOLATE_ID&gt;/&lt;ISOLATE_ID&gt;.dcn.fa</b></li>
<li>Summary of assemblies without contaminant contigs <b>results/quast/AssemblyMetrics.dcn.tsv</b></li>
</ul>
</p>

<h2>1 BLAST CONTIGS TO nt DATABASE</h2>
<p>In order to blast contigs to nt database, the nt database may be downloaded. Otherwise, the blast needs to use the nt database remotely (See Blast manual for this). If downloading the nt database to a scinet location, make sure to use the datamover node of scinet. We employed Blast v. 2.8.1<sup>1</sup></p>
<h3>1.1 RETRIEVE NCBI's nt DATABASE</h3>

In [ ]:
# Go to database folder
cd data/database/ncbi

In [ ]:
#Download nt database. This will take a long time and use a lot of bandwidth.
../../bin/ncbi-blast-2.8.1+/bin/update_blastdb.pl --decompress nt

<h3>1.2 PERFORM BLAST</h3>

In [ ]:
#Go to the decontamination folder
cd ../../../results/decontamination/

<h4>1.2a SUBMIT TO SCINET</h4>

In [ ]:
#Create blast jobs
for i in `cat ../../doc/IdList.txt`;do echo -e '#!/bin/bash'"\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=40\n#SBATCH --time=3:00:00\n#SBATCH --job-name=deconBlast_jobs-$i\n#SBATCH --output=deconBlast_job-$i\nexport BLASTDB=../../data/database/ncbi/" > deconBlastJob_$i.sh; echo ../../bin/ncbi-blast-2.8.1+/bin/blastn -query ../spades/$i/$i.fa -outfmt "'"10 qacc sacc pident length mismatch gapopen qstart qend sstart send evalue bitscore staxids sscinames scomnames sblastnames sskingdoms"'" -db nt -num_threads 40 -out $i".blast" >> deconBlastJob_$i.sh; done

In [ ]:
#Submit blast jobs
for i in `cat ../../doc/IdList.txt`; do sbatch deconBlastJob_$i.sh;done

<h4>1.2b OR RUN LOCALLY (THIS WILL TAKE A LONG TIME)</h4>

In [ ]:
#Set datahase system variable
export BLASTDB=../../data/database/ncbi/

In [ ]:
#Blast each contig against nt database
for i in `cat ../../doc/IdList.txt`;do ../../bin/ncbi-blast-2.8.1+/bin/blastn -query ../spades/$i/$i.fa -outfmt '10 qacc sacc pident length mismatch gapopen qstart qend sstart send evalue bitscore staxids sscinames scomnames sblastnames sskingdoms' -db nt -num_threads 4 -out $i".blast"; done

<h2>2 MAPPING SEQUENCING READS TO Spades ASSEMBLY</h2>
<p>The sequencing reads of each isolate are mapped to the contigs of their respective assemblies. This step is performed in order to figure out which sequencing reads are part of which contigs. We employed BWA v.0.17<sup>2</sup></p>

<h3>2a SUBMIT TO SCINET</h3>

In [ ]:
#Create BWA jobs
for i in `cat ../../doc/IdList.txt`;do echo -e '#!/bin/bash'"\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=40\n#SBATCH --time=1:00:00\n#SBATCH --job-name=deconBwa_jobs-$i\n#SBATCH --output=deconBwa_job-$i" > deconBwaJob_$i.sh; echo ../../bin/bwa-0.7.17/bwa index ../../results/spades/$i/$i.fa >> deconBwaJob_$i.sh; echo ../../bin/bwa-0.7.17/bwa mem -t 40 ../../results/spades/$i/$i.fa ../dedupper/$i"_1.ddp.fq" ../dedupper/$i"_2.ddp.fq  > "$i".sam" >> deconBwaJob_$i.sh; done

In [ ]:
#Submit bwa job to scinet
for i in `cat ../../doc/IdList.txt`;do sbatch deconBwaJob_$i.sh; done

<h3>2b OR RUN LOCALLY (THIS WILL TAKE A LONG TIME)</h3>

In [ ]:
#Index the Spades assembly and map reads to that genome
for i in `cat ../../doc/IdList.txt`; do ../../bin/bwa-0.7.17/bwa index ../../results/spades/$i/$i.fa; echo ../../bin/bwa-0.7.17/bwa mem -t 40 ../../results/spades/$i/$i.fa ../dedupper/$i"_1.ddp.fq" ../dedupper/$i"_2.ddp.fq"  > $i.sam; done

<h2>3 PARSE BLAST RESULTS</h2>
<p>This step checks if the contig has a blast hit in the Burkholderia genus. It identifies non-Burkholderia contigs and marks them appropriately. Once this step is completed the results can be plotted with the R jupyter notebook "<b>4a_CoverageOverLengthPlots</b>".</p>

In [ ]:
# Identify which contigs map to a genus other than Burkholderia
for i in `cat ../../doc/IdList.txt`;do ../../bin/blast_parser.py $i".blast" "../spades/"$i"/AssemblyMetrics.tsv" Burkholderia "ParsedAssemblyMetrics_"$i".csv";done

<h2>4 IDENTIFY CONTIGS TO BE REMOVED</h2>
<p>This step creates a list of contigs which have a coverage lower than 10 and that have been flagged as not from the desired genus. The input HAS to be the "<b>ParsedAssemblyMetrics_*.csv</b>" file, which is the result from the previous step.</p>

In [ ]:
for i in `cat ../../doc/IdList.txt`; do ../../bin/contig_localizer.py "ParsedAssemblyMetrics_"$i".csv" 10 "ContigsToBeRemoved_"$i".txt";done

<h2>5 INDENTIFY READS TO BE REMOVED</h2>
<p>This step identifies reads that map to the contigs to be removed and creates a list of these reads. The input has to be the "<b>ContigsToBeRemoved_*.txt</b>" file, which is result from the previous step.</p>

In [ ]:
for i in `cat ../../doc/IdList.txt`;do ../../bin/read_localizer.py "ContigsToBeRemoved_"$i".txt" $i".sam" "ReadsToBeRemoved_"$i".txt";done

<h2>6 REMOVED FLAGGED READS</h2>
<p>This removes the reads found in the list "<b>ReadsToBeRemoved_*.txt</b>", which is produced in the previous step.</p>

In [ ]:
for i in `cat ../../doc/IdList.txt`;do ../../bin/read_remover.py "ReadsToBeRemoved_"$i".txt" "../dedupper/"$i"_1.ddp.fq" "../dedupper/"$i"_2.ddp.fq" $i"_1.dcn.fq" $i"_2.dcn.fq" ;done

<h2>7 SUMMARIZE</h2>
<p>Record the number of reads and bases left over after removing potential contamination reads.</p>

In [ ]:
echo -e "IsolateId\tReadOneCount\tReadTwoCount\tBaseCountOne\tBaseCountTwo" > ContaminationRemovalSummary.tsv 
for i in `cat ../../doc/IdList.txt`; do RN_1=`awk -v I=$i 'NR%4==1' $i"_1.dcn.fq" | wc -l`;RN_2=`awk -v I=$i 'NR%4==1' $i"_2.dcn.fq" | wc -l`;BC_1=`awk -v I=$i 'NR%4==2 {printf $1}' $i"_1.dcn.fq" | wc -m`;BC_2=`awk -v I=$i 'NR%4==2 {printf $1}' $i"_2.dcn.fq"| wc -m`; echo -e $i"\t"$RN_1"\t"$RN_2"\t"$BC_1"\t"$BC_2; done >> ContaminationRemovalSummary.tsv

Remove putative contaminant contigs from their respective <i>de novo</i> assemblies.

In [ ]:
for i in `cat ../../doc/IdList.txt`;do ../../bin/contig_remover.py "ContigsToBeRemoved_"$i".txt" "../../results/spades/"$i"/"$i".fa" "../../results/spades/"$i"/"$i".dcn.fa"; don

Calculate metrics on the assemblies given the removed contigs. We employed quast v 5.0.2<sup>3</sup>.

In [ ]:
# Go into the quast directory
cd ../quast

# Executing quast
for i in `cat ../../doc/IdList.txt`;do ../../bin/quast-5.0.2/quast.py -m 1000 ../spades/$i/$i.dcn.fa -o $i.dcn; done

# Summarizing assembly metrics
echo -e "Isolate\tContigCount\tTotalLength\tN50" >> AssemblySummary.dcn.tsv
for i in `cat ../../doc/IdList.txt`;do contig=`sed '5q;d' $i.dcn/report.txt | awk '{print $6}'`; length=`sed '11q;d' $i.dcn/report.txt | awk '{print $6}'`; n50=`sed '20q;d' $i.dcn/report.txt | awk '{print $2}'`; echo -e $i"\t"$contig"\t"$length"\t"$n50;done >> AssemblySummary.dcn.tsv

<h4>NEXT STEPS</h4>
<p>Quality control of reads. Additionally the new assembly metrics (post contamination removal) can be plotted using the script in "<b>4b_PostDecontaminationAssemblyPlots</b>".</p>

<sup>1</sup> https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=Download
<br><sup>2</sup> http://bio-bwa.sourceforge.net
<br><sup>3</sup> http://quast.bioinf.spbau.ru